In [18]:
import numpy as np
import pandas as pd
from pandas import read_csv
from datetime import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib
import math

In [19]:
date_parser = lambda d: datetime.strptime(d, '%Y%m')

# Read Dataset from CSV into Pandas Dataframe.
indexed_data_df = read_csv(
    '../../data/data.csv',
    names=['date', 'temperature'],
    dtype={'date': 'str', 'temperature': 'float'},
    parse_dates=['date'],
    date_parser=date_parser,
    index_col='date'
)

data_df = read_csv(
    '../../data/data.csv',
    names=['date', 'temperature'],
    dtype={'date': 'str', 'temperature': 'float'},
    parse_dates=['date'],
    date_parser=date_parser
)

In [22]:
# Isolate dataset trend only and save it as a numpy array
decompose_res = seasonal_decompose(indexed_data_df, model='additive', freq=12)
trend_data_np = np.array([[d, round(t, 5)] for d, t in zip(data_df['date'].to_numpy(), decompose_res.trend) if not math.isnan(t)])